# バッチ推論サービスを作成する

以前のラボでは、Azure ML *パイプライン* を使用してモデルのトレーニングと登録を自動化し、モデルをリアルタイムで*推論*するための Web サービスとして公開しました (モデルから予測を取得する)。次に、これら 2 つの概念を組み合わせて、*バッチ推論*用パイプラインを作成します。それは何を意味しますか? さて、健康クリニックは一日中患者の測定を取り、各患者の詳細を別々のファイルに保存すると想像してください。その後、一晩で糖尿病予測モデルを使用して、その日のすべての患者データをバッチとして処理し、翌朝待つ予測を生成し、糖尿病のリスクがあると予測される患者をフォローアップできるようにします。この演習では、この手順を実装します。

## ワークスペースに接続する

まず、Azure ML SDK を使用してワークスペースに接続します。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## モデルをトレーニングして登録する

このコースの前のラボを完了している場合は、**diasia_model** モデルの多くのバージョンを登録しているので、すぐに使用できます。

まだ完了していない場合は、下のセルを実行してモデルをトレーニングし、登録できます。

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# ワークスペースで Azure 実験を作成する
experiment = Experiment(workspace = ws, name = "diabetes-training")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# 糖尿病データセットを読み込む
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# フィーチャーとラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# デシジョン ツリー モデルをトレーニングする
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# トレーニング済みモデルを保存する
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# 実行を完了する
run.complete()

# モデルを登録する
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

## バッチデータを生成およびアップロードする

このコースの新しいデータを取得する患者を含む完全なスタッフの診療所は実際にないので、糖尿病 CSV ファイルからランダムなサンプルを生成し、それらを使用してパイプラインをテストします。次に、そのデータを Azure Machine Learning ワークスペースのデータストアにデータをアップロードし、データセットを登録します。

In [ ]:
from azureml.core import Datastore, Dataset
import pandas as pd
import os

# 糖尿病データを読み込む
diabetes = pd.read_csv('data/diabetes2.csv')
# フィーチャー列の 100 項目のサンプルを取得する (糖尿病ラベルではない)
sample = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].sample(n=100).values

# フォルダーを作成する
batch_folder = './batch-data'
os.makedirs(batch_folder, exist_ok=True)
print("Folder created!")

# 各サンプルを個別のファイルとして保存する
print("Saving files...")
for i in range(100):
    fname = str(i+1) + '.csv'
    sample[i].tofile(os.path.join(batch_folder, fname), sep=",")
print("files saved!")

# 既定のデータストアにファイルをアップロードする
print("Uploading files to datastore...")
default_ds = ws.get_default_datastore()
default_ds.upload(src_dir="batch-data", target_path="batch-data", overwrite=True, show_progress=True)

# 入力データ用データセットを登録する
batch_data_set = Dataset.File.from_files(path=(default_ds, 'batch-data/'), validate=False)
try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='batch-data',
                                             description='batch data',
                                             create_new_version=True)
except Exception as ex:
    print(ex)

print("Done!")

## コンピューティングを作成する

パイプラインのコンピューティング コンテキストが必要なので、前の演習で使用した Azure ML コンピューティング クラスターを使用します (まだ存在しない場合は作成されます)。

> **重要**: 実行する前に、*コンピューティング クラスター*を以下のコードでコンピューティング クラスターの名前に変更してください。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

# 既存のクラスターを確認する
try:
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # AzureMl コンピューティング リソースを作成する
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=2)
    inference_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

inference_cluster.wait_for_completion(show_output=True)

## バッチ推論用パイプラインを作成する

これで、バッチ推論に使用するパイプラインを定義する準備が整いました。パイプラインではバッチ推論を実行するために Python コードが必要となるので、パイプラインで使用されるすべてのファイルを保存できるフォルダーを作成しましょう。

In [ ]:
import os
# 実験ファイル用フォルダーを作成する
experiment_folder = 'batch_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

次に、実際の作業を行う Python スクリプトを作成し、パイプライン フォルダーに保存します。

In [ ]:
%%writefile $experiment_folder/batch_diabetes.py
import os
import numpy as np
from azureml.core import Model
import joblib


def init():
    # パイプライン手順が初期化されたときに実行する
    global model

    # モデルを読み込む
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)


def run(mini_batch):
    # これはバッチごとに実行する
    resultList = []

    # バッチ内の各ファイルを処理する
    for f in mini_batch:
        # カンマ区切りデータを配列に読み込む
        data = np.genfromtxt(f, delimiter=',')
        # 予測のために 2 次元配列に変形する (モデルは複数の項目を必要とします)
        prediction = model.predict(data.reshape(1, -1))
        # 結果に予測をアペンドする
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

次に、スクリプトで必要な依存関係を含む実行コンテキストを定義します

In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.runconfig import CondaDependencies

# モデルに必要な依存関係を追加する
# Scikit-learn モデルの場合、Scikit-learn が必要です
# 並列パイプラインの手順では、azureml コアと azureml-dataprep[fuse] が必要です
cd = CondaDependencies.create(pip_packages=['scikit-learn','azureml-core', 'azureml-dataprep[fuse]'])

batch_env = Environment(name='batch_environment')
batch_env.python.conda_dependencies = cd
batch_env.docker.enabled = True
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

パイプラインを使用して、バッチ予測スクリプトを実行し、入力データから予測を生成し、結果をテキスト ファイルとして出力フォルダーに保存します。これを行うには、**ParallelRunStep** を使用して、バッチ データを並列処理し、結果を *parallel_run_step.txt* という名前の単一の出力ファイルに照合することができます。

In [ ]:
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.pipeline.core import PipelineData

default_ds = ws.get_default_datastore()

output_dir = PipelineData(name='inferences', 
                          datastore=default_ds, 
                          output_path_on_compute='diabetes/results')

parallel_run_config = ParallelRunConfig(
    source_directory=experiment_folder,
    entry_script="batch_diabetes.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=inference_cluster,
    node_count=2)

parallelrun_step = ParallelRunStep(
    name='batch-score-diabetes',
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('diabetes_batch')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

次は、ステップをパイプラインに入れて実行します。

> **注**: これには時間がかかる場合があります。

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
pipeline_run = Experiment(ws, 'batch_prediction_pipeline').submit(pipeline)
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

パイプラインの実行が終了すると、結果の予測は、パイプラインの最初の (そして唯一の) ステップに関連付けられた実験の出力に保存されます。次のように取得できます。

In [ ]:
import pandas as pd
import shutil

shutil.rmtree('diabetes-results', ignore_errors=True)

prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='diabetes-results')


for root, dirs, files in os.walk('diabetes-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# クリーンアップ出力形式
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# 最初の 20 件の結果を表示する
df.head(20)

## パイプラインを公開し、その REST インターフェイスを使用する

バッチ推論用の作業パイプラインができたので、それを公開し、REST エンドポイントを使用してアプリケーションから実行できます。

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name='Diabetes_Parallel_Batch_Pipeline', description='Batch scoring of diabetes data', version='1.0')

published_pipeline

公開されたパイプラインにはエンドポイントがあり、Azure portal で確認できます。また、公開されたパイプライン オブジェクトのプロパティとして見つけることもできます。

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

エンドポイントを使用するには、クライアント アプリケーションが HTTP 経由で REST 呼び出しを行う必要があります。この要求は認証される必要があるため、Authorization ヘッダーが必要です。これをテストするには、現在の接続から Azure ワークスペースへの Authorization ヘッダーを使用します。これは、次のコードを使用して取得できます。

> **注**: 実際のアプリケーションには、認証に使用するサービス プリンシパルが必要です。

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print('Authentication header ready.')

これで、REST インターフェイスを呼び出す準備ができました。パイプラインは非同期で実行されるため、識別子を取得します。実行中のパイプライン実験を追跡するために使用できます。

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "Batch_Pipeline_via_REST"})
run_id = response.json()["Id"]
run_id

実行 ID があるため、**RunDetails** ウィジェットを使用して、実行中の実験を表示できます。

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["Batch_Pipeline_via_REST"], run_id)
RunDetails(published_pipeline_run).show()

前と同様に、結果は最初のパイプライン ステップの出力にあります。

In [ ]:
import pandas as pd
import shutil

shutil.rmtree("diabetes-results", ignore_errors=True)

prediction_run = next(published_pipeline_run.get_children())
prediction_output = prediction_run.get_output_data("inferences")
prediction_output.download(local_path="diabetes-results")


for root, dirs, files in os.walk("diabetes-results"):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# クリーンアップ出力形式
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# 最初の 20 件の結果を表示する
df.head(20)

これで、毎日の患者データをバッチ処理するために使用できるパイプラインが作成されました。

**詳細情報**: バッチ推論用パイプラインの使用方法の詳細については、Azure Machine Learning のドキュメントの[「バッチ予測の実行方法」](https://docs.microsoft.com/azure/machine-learning/how-to-run-batch-predictions)を参照してください。